In [7]:
import sys

In [8]:
print(sys.path)

['/Users/nmolivo/Documents/Repos/case_scrape/notebooks', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/nmolivo/Documents/Repos/case_scrape/.venv/lib/python3.10/site-packages', '/Users/nmolivo/Documents/Repos/case_scrape']


In [9]:
sys.path.append("/Users/nmolivo/Documents/Repos/case_scrape")

In [10]:
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker, scoped_session
from chromedriver_py import binary_path
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException#, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy.util.compat import contextmanager
import random
from typing import Tuple
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

from case_scrape.models import Base
from case_scrape.db.db_config import DBConfig
from case_scrape import models

In [13]:
from case_scrape.scripts.web_scrape import *

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010b327fa8 chromedriver + 4886440
1   chromedriver                        0x000000010b2a5643 chromedriver + 4351555
2   chromedriver                        0x000000010aef3b27 chromedriver + 477991
3   chromedriver                        0x000000010af3876c chromedriver + 759660
4   chromedriver                        0x000000010af389c1 chromedriver + 760257
5   chromedriver                        0x000000010af7c204 chromedriver + 1036804
6   chromedriver                        0x000000010af5e5fd chromedriver + 914941
7   chromedriver                        0x000000010af795fe chromedriver + 1025534
8   chromedriver                        0x000000010af5e3a3 chromedriver + 914339
9   chromedriver                        0x000000010af2857f chromedriver + 693631
10  chromedriver                        0x000000010af29b1e chromedriver + 699166
11  chromedriver                        0x000000010b2f4b9e chrome

In [ ]:
opts = Options()
# opts.headless = True
service = ChromeService(executable_path = ChromeDriverManager().install())
driver = webdriver.Chrome(options = opts, service=service)

In [7]:
case_no_str = "655783"

In [8]:
driver.get("https://cpdocket.cp.cuyahogacounty.us/Search.aspx")


In [9]:
driver = search_case_number(driver, case_no_str)

In [10]:
driver, case_number = fetch_case_summary(session, driver)

In [11]:
driver, charge_df = get_table(
        driver,
        case_number,
        header_xpath="//table[(@id = 'SheetContentPlaceHolder_caseCharges_gvCharges')]//th",
        data_xpath="//table[(@id = 'SheetContentPlaceHolder_caseCharges_gvCharges')]//td",
    )

In [12]:
charge_df

,TYPE,STATUTE,CHARGE DESCRIPTION,DISPOSITION,CASE_NUMBER
0,INDICT,2911.13.A,BREAKING AND ENTERING,PLD GLTY- N/C - IND @ P/T,CR-21-655783-A
1,INDICT,2909.05.B(1)(B),VANDALISM,PLD GLTY- N/C - IND @ P/T,CR-21-655783-A
2,INDICT,2913.02.A(1),THEFT; AGGRAVATED THEFT,NOLLE,CR-21-655783-A
3,INDICT,2921.31.A,OBSTRUCTING OFFICIAL BUSINESS,NOLLE,CR-21-655783-A


In [13]:
engine = DBConfig(
    password=os.getenv("DB_PASSWORD"), user=os.getenv("DB_USER")
).get_engine()

Session = sessionmaker(bind=engine)
session = Session()
db = session

In [14]:
from sqlalchemy import delete

In [15]:
dele = models.Charge.__table__.delete().where(models.Charge.case_number == case_number)

In [16]:
db.execute(dele)
db.commit()

In [17]:
len(db.query(models.Charge).filter(models.Charge.case_number ==case_number).all())

0